<hr>

# Clustering Mnist using TransferLearning

In this notebook we will explore:

1. Building of a autoencoder architecture.
2. Training.
3. Encoder Prediction.
4. Kmeans
5. NMI metrics

John Branch: jwbranch@unal.edu.co

Carlos Madrigal camadrigalg@unal.edu.co - cmadrigal@grupodot.com


# Import Libraries

In [2]:
from keras.datasets import mnist

In [3]:
%pylab inline
import os
import keras
import numpy as np
import pandas as pd
import keras.backend as K
import tensorflow as tf
from time import time
from sklearn.cluster import KMeans
from keras import callbacks
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Dense, Input
from keras.initializers import VarianceScaling
from keras.engine.topology import Layer, InputSpec
from sklearn.metrics import accuracy_score, normalized_mutual_info_score
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions
from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPool2D, Dense, Softmax
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential

Populating the interactive namespace from numpy and matplotlib


# Load DataSet

In [4]:
(train_x, train_y), (val_x, val_y) = mnist.load_data()

# Data Normalization

In [5]:
train_x = train_x/255.
val_x = val_x/255.

# Load Pretrained Model

In [6]:
HEIGHT, WIDTH, CHANNELS = (128,128,3)
input_t = keras.Input(shape=(HEIGHT, WIDTH, CHANNELS)) 
transfer_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_t)

transfer_model.trainable = False
transfer_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

# Transfer Learning

In [20]:
middle = transfer_model.get_layer('conv5_block2_out')
tl_model = Model(inputs = input_t, outputs=middle.output)
tl_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

# Prediction

In [21]:
train_x_new = np.stack((train_x,)*3, axis=-1)
val_x_new = np.stack((val_x,)*3, axis=-1)


In [23]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input

prediction_train = tl_model.predict(preprocess_input(train_x_new))
prediction_train = GlobalAveragePooling2D()(prediction_train)

prediction_val = tl_model.predict(preprocess_input(val_x_new))
prediction_val = GlobalAveragePooling2D()(prediction_val)

prediction_train.numpy().shape, prediction_val.numpy().shape

((60000, 2048), (10000, 2048))

In [24]:
prediction_train_sample= prediction_train.numpy()[:,list(np.arange(0,2048,3))]
prediction_val_sample= prediction_val.numpy()[:,list(np.arange(0,2048,3))]
prediction_train_sample.shape

(60000, 683)

# Kmeans

In [25]:
kmeans = KMeans(n_jobs=-1, n_clusters=10, n_init=10)
kmeans.fit(prediction_train_sample)
pred_val = kmeans.predict(prediction_val_sample)
pred_train = kmeans.predict(prediction_train_sample)

In [28]:
metrics_NMI_val=normalized_mutual_info_score(val_y, pred_val)
metrics_NMI_train=normalized_mutual_info_score(train_y, pred_train)
print("Validation NMI metrics: ", metrics_NMI_val)
print("Train NMI metrics: ", metrics_NMI_train)

<hr>

# Lets work!

1. Select different layers on Resnet.
2. Try to load CIFAR10 and cluster it.
